In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# author:
import pickle, os, json, argparse, sys, time
from Extkinter import *
from PIL import Image,ImageTk
from datetime import datetime
import pandas as pd
from tkinter import *
import tkinter as tk
from tkinter import ttk
import tkinter.messagebox
import time
import numpy as np
np.seterr(divide='ignore', invalid='ignore')

from myEncoder.Encode import VitalSigns2Digital
import Pkuseg  # 专业领域分词
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

from keras.utils import to_categorical
import tensorflow as tf
from tensorflow.compat.v1 import InteractiveSession
tf.keras.backend.clear_session()
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

from TransNet import TransNet
import pyttsx3
import threading
import warnings
warnings.filterwarnings("ignore")
os.environ["CUDA_VISIBLE_DEVICES"] = "0" or "1"
tok = Tokenizer()
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

Using TensorFlow backend.


In [2]:
def InputBox(default_value, x, y):
    entry = ttk.Entry(window, width=10)
    entry.place(x=x, y=y)
    entry.insert(0, default_value)

def InputBoxChiefComplaintRef(default_value, x, y):
    entry = ttk.Entry(window,width=30)
    entry.place(x=x, y=y)
    entry.insert(0, default_value)
    
def AgeTranslation(birth):
    if birth != '空值' and birth != '':
        if "/" in birth:
            birth_d = datetime.strptime(birth, "%Y/%m/%d")
        elif "-" in birth:
            birth_d = datetime.strptime(birth, "%Y-%m-%d").date()
        else:
            birth_d = datetime.strptime(birth, "%Y%m%d").date()
        today_d = datetime.now()
        age = today_d.year - birth_d.year
        return age
    else:
        return '空值'

def update_patient_info():
    global idx, Name, ID, Gender, Age, Temp, PulseRate, Respiration
    global BloodPres, SpO2, ArrMode, ChiefComp, EndIndex

    if idx < EndIndex:
        Name = data['病人姓名'][idx]
        ID = data['病人ID'][idx]
        Gender = data['性别'][idx]
        Age = AgeTranslation(data['出生日期'][idx])
        Temp = data['T℃'][idx]
        PulseRate = data['P(次/分)'][idx]
        Respiration = data['R(次/分)'][idx]
        BloodPres = data['BP(mmHg)'][idx]
        SpO2 = data['SpO2'][idx]
        ArrMode = data['到院方式'][idx]
        ChiefComp = data['分诊印象'][idx]

        # 更新GUI中的患者信息
        InputBox(Name, 160, 0)
        InputBox(ID, 160, 30)
        InputBox(Gender, 160, 60)
        InputBox(Age, 160, 90)
        InputBox(Temp, 60, 200)
        InputBox(PulseRate, 60, 230)
        InputBox(Respiration, 60, 260)
        InputBox(BloodPres, 60, 290)
        InputBox(SpO2, 60, 320)
        InputBox(ArrMode, 400, 0)
        name_refer.delete(1.0, tk.END)
        name_refer.insert(tk.END, ChiefComp)
        LevelFinal = LevelModeLabel.get()
        DepartmentFinal = DepartmentLabel.get()        
    else:
        tk.messagebox.showinfo("提示", "已经到达数据末尾，将关闭窗口。")
        window.destroy()  # 关闭窗口

def creat_csv(file_path, COLUMN):
    if not os.path.exists(file_path):
        tabel = pd.DataFrame(columns=COLUMN)
        tabel.to_csv(file_path, index=False, header=True, encoding="utf_8_sig")
    else:
        tabel = pd.read_csv(file_path, engine='python')
    return tabel

def data_input(self, COLUMN, variables):
    data = pd.DataFrame(columns=COLUMN)
    for idx, i in zip(COLUMN, variables):
        data.at[0, idx] = i
    VS = [np.hstack(([to_categorical(self.func(data[i].values[0], i), LIB2[i]) for i in Parems]))]
    IM = self.unstructure(data)
    Data = np.hstack([VS, IM])
    return [VS], Data

def ResultFinal(RevisedDictionary, y_predict1, y_predict2):
    if '入抢救室' in RevisedDictionary:
        y_predict1 = '一级'
        RevisedDictionary = ['入抢救室']
    elif len(RevisedDictionary) == 0:
        RevisedDictionary = [y_predict2]
    else:
        RevisedDictionary.append(y_predict2)
        RevisedDictionary = np.unique(RevisedDictionary)
        if len(RevisedDictionary) == 1:
            RevisedDictionary = RevisedDictionary[0]
    return y_predict1, RevisedDictionary

def Conditional_judgment(WhitePath, Chief_Complaint):
    DepartWhiteList = os.listdir(WhitePath)
    Result = []
    for k in DepartWhiteList:
        Path = os.path.join(WhitePath, k)
        whitelist = pd.read_csv(Path, header=None)
        include = []
        for i in whitelist[0]:
            include.append(i)
        if any(word if word in Chief_Complaint else False for word in include):
            Result.append(k[:-4])
        else:
            pass
    return Result

def AITriageModule():
    global idx, ID, Name, Gender, Age, Temp, PulseRate, Respiration
    global BloodPres, SpO2, ArrMode, ChiefComp
    Time = time.strftime('%Y/%m/%d %X', time.localtime())
    Triage_Time = Time
    Variables = [Gender, data['出生日期'][idx], ArrMode, Temp, PulseRate, Respiration, BloodPres, SpO2]
    Variables = ['空值' if str(i).isspace() or len(str(i)) == 0 else i for i in Variables]
    ColumnsTransfer = [ '性别', '出生日期','到院方式', 'T℃', 'P(次/分)', 'R(次/分)', 'BP(mmHg)', 'SpO2']
    
    data_VSandIM = pd.DataFrame(columns=ColumnsTransfer)
    for ind, i in zip(ColumnsTransfer, Variables):
        data_VSandIM.at[0, ind] = i    
    VS = [np.hstack(([to_categorical(func(data_VSandIM[i].values[0], i), LIB2[i]) for i in ColumnsTransfer]))]
    IM = Unstructure(ChiefComp)
    DataVSandIM = np.hstack([VS, IM])
    predictions = model.predict([[VS], DataVSandIM])
    y_predict1 = dicLevel[np.argmax(predictions[0])]
    y_predict2 = dicDepart[np.argmax(predictions[1])]
    
    RevisedDictionary = Conditional_judgment(WhitePath, ChiefComp)
    Level, Department = ResultFinal(RevisedDictionary, y_predict1, y_predict2)
    return Level, Department

def func(x, Name):
    funcs = LIB1[Name]
    result = getattr(VitalSigns2Digital(), funcs)(x)
    return result

def SegmentFunc(content, stopwords):
    segment = []
    segs = segcut.cut(content)
    for seg in segs:
        if seg not in stopwords:
            segment.append(seg)
    return segment

def Unstructure(ChiefComplaint):
    im = SegmentFunc(ChiefComplaint, exclude)
    tok.fit_on_texts([im])
    with open('./JSON/word_index.json', 'r') as f:
        word_index = json.load(f)
    tok.word_index = word_index
    x = tok.texts_to_sequences([im])
    X = sequence.pad_sequences(x, maxlen=13)
    return X

def button_click():
    global counter, idx, Consumption, start, end
    global ID, Name, Gender, Age, Temp, PulseRate, Respiration
    global BloodPres, SpO2, ArrMode, ChiefComp
        
    SingleData=[] 
    label.config(text=str(counter))
    
    Level_Verify = LevelModeLabel.get()
    Department_Verify = DepartmentLabel.get()
    
    end = time.time()
    Consumption = np.round((end - start), 3)
    start = time.time()
    
    SingleData = [ID, Name, Gender, data['出生日期'][idx], ArrMode, ChiefComp, Temp,PulseRate, 
                  Respiration, BloodPres, SpO2,data['M分诊耗时(秒)'][idx], data['级别'][idx], 
                  data['去向'][idx], data['M级别'][idx], data['M科室'][idx], Consumption, Level_Verify, Department_Verify]
    Write(save_path, SingleData, INDEX)
    
    counter += 1
    idx = counter
    if idx > EndIndex-1:
        tk.messagebox.showinfo("提示", "已经到达数据末尾，将关闭窗口。")
        window.destroy()
    else:
        update_patient_info()
        Level, Department = AITriageModule()
        LevelModeLabel.set(Level)
        DepartmentLabel.set(Department)
#         DepartmentLabel.insert(tk.END, Department)#科室推荐换行
        
def Write(save_path, variables, columns):
    try:
        DATA = pd.read_csv(save_path)
    except FileNotFoundError:
        DATA = pd.DataFrame(columns=columns)
    new_row = pd.Series(variables, index=columns)
    DATA= pd.concat([DATA, new_row.to_frame().transpose()], ignore_index=True)
    DATA.to_csv(save_path, index=False,encoding="utf_8_sig")

In [3]:
if __name__ == '__main__':
    # 全局变量
    window = None
    data = None
    Name, ID, Gender, Age, Temp, PulseRate, Respiration, BloodPres, SpO2, ArrMode, ChiefComp, LevelFinal, DepartmentFinal= [None] * 13
    name_refer = None  # 声明为全局变量
    
    window = tk.Tk()
    window.title('智能分诊系统v1.0-测试版本')
    window.geometry('620x400')
    window.resizable(0, 0)
    
    path = r"./data/Data.txt"
    NameDoctor = input('请输入分诊人员的姓名（建议中文）：')
    StartIndex = int(input('请输入起始索引(比如从1开始)：'))
    if StartIndex == 0:
        StartIndex = 1
    EndIndex = int(input('请输入结束索引（比如到100结束）：'))
    ResultPath = r'./result'
    if not os.path.exists(ResultPath):
         os.makedirs(ResultPath)
    save_path = os.path.join(ResultPath, NameDoctor + '_TriageData.csv')
    
    
    WhitePath = r'./StopWords/whitelist'
    
    Columns = ['病人ID','病人姓名','性别','出生日期','到院方式','分诊印象','T℃', 'P(次/分)',
               'R(次/分)', 'BP(mmHg)', 'SpO2', 'M分诊耗时(秒)', '级别', '去向', 'M级别', 'M科室', 'AI分诊耗时(秒)', '级别AI','科室AI']
    Data = pd.read_csv(path, engine='python')
    
    P = pd.DataFrame(Data) #, columns=Columns
    if EndIndex>len(Data):
        EndIndex = len(Data)
    data = P[StartIndex-1: EndIndex]
    data = data.fillna(value='空值')
    Tabel = creat_csv(save_path, Columns)
    INDEX = Columns    
    HistoryData = pd.read_csv(save_path, engine='python')
    if   StartIndex >= len(HistoryData):#数据右边
        counter = StartIndex-1
    elif 0<= StartIndex<len(HistoryData) and EndIndex>len(HistoryData):
        counter = len(HistoryData)
    elif EndIndex <=len(HistoryData):#数据之间
#         tk.messagebox.showinfo("提示", "该部分已处理过，将关闭窗口。")
#         window.destroy()
        print('该部分已处理过！！')
        counter = len(HistoryData)
#         os._exit(0)# 括号要放值os._exit(0)--正常退出
    idx = counter
    
    Name = data['病人姓名'][idx]
    ID = data['病人ID'][idx]
    Gender = data['性别'][idx]
    Age = AgeTranslation(data['出生日期'][idx])

    Temp = data['T℃'][idx]
    PulseRate = data['P(次/分)'][idx]
    Respiration = data['R(次/分)'][idx]
    BloodPres = data['BP(mmHg)'][idx]
    SpO2 = data['SpO2'][idx]
    ArrMode = data['到院方式'][idx]
    ChiefComp = data['分诊印象'][idx]
    name_refer = tk.Text(window, width=30, height=3, wrap=tk.WORD)
    name_refer.insert(tk.END, ChiefComp)
    name_refer.place(x=400, y=100)

    # *************************************患者信息***********************
    # *************************************患者信息***********************
    tk.Label(window, text="", bg="lightgray", fg="white").place(x=0, y=0, width=250, height=140)
    tk.Label(window, text="患者信息", bg="lightblue", fg="white", font=("华文细黑", 12)).place(x=15, y=165)

    # 第4步，加载 wellcome image
    image_text = Text(window, height=10, width=16)
    photo = PhotoImage(file="RC.png")
    image_text.image_create('0.0', image=photo)
    image_text.grid(row=2, column=1, columnspan=2, sticky=W)

    # 第5步
    tk.Label(window, text='姓名:', bg="lightgray", font=('华文细黑', 12)).place(x=120, y=0)
    InputBox(Name, 160, 0)
    tk.Label(window, text='ID:', bg="lightgray", font=('华文细黑', 12)).place(x=120, y=30)
    InputBox(ID, 160, 30)
    tk.Label(window, text='性别:', bg="lightgray", font=('华文细黑', 12)).place(x=120, y=60)
    InputBox(Gender, 160, 60)
    # # 单选钮，男
    # def show():
    #     print(Gender[sex.get()-1][0])
    # sex = tk.IntVar(value=1)
    # Gender = [("男", 0),("女", 1),]
    # radioMan = tk.Radiobutton(window, variable=sex, value=1, text='男', bg="lightgray", font=('华文细黑', 12),command=show)
    # radioMan.place(x=160, y=60, width=50, height=20)
    # radioWoman = tk.Radiobutton(window, variable=sex, value=2, text='女', bg="lightgray", font=('华文细黑', 12),command=show)
    # radioWoman.place(x=210, y=60, width=70, height=20)

    tk.Label(window, text='年龄:', bg="lightgray", font=('华文细黑', 12)).place(x=120, y=90)
    InputBox(Age, 160, 90)

    # 第6步，*************************************生命体征
    # 第6步，*************************************生命体征
    tk.Label(window, text="", bg="darkseagreen", fg="black").place(x=0, y=160, width=160, height=200)
    tk.Label(window, text="生命体征", bg="olivedrab", fg="white", font=("华文细黑", 12)).place(x=0, y=165)
    tk.Label(window, text='体温:', bg="darkseagreen", fg="black", font=('华文细黑', 12)).place(x=15, y=200)
    InputBox(Temp, 60, 200)
    tk.Label(window, text='心率:', bg="darkseagreen", fg="black", font=('华文细黑', 12)).place(x=15, y=230)
    InputBox(PulseRate, 60, 230)
    tk.Label(window, text='呼吸:', bg="darkseagreen", fg="black", font=('华文细黑', 12)).place(x=15, y=260)
    InputBox(Respiration, 60, 260)
    tk.Label(window, text='血压:', bg="darkseagreen", fg="black", font=('华文细黑', 12)).place(x=15, y=290)
    InputBox(BloodPres, 60, 290)
    tk.Label(window, text='血氧:', bg="darkseagreen", fg="black", font=('华文细黑', 12)).place(x=15, y=320)
    InputBox(SpO2, 60, 320)

    # *************************************到院方式
    # *************************************到院方式
    ArrivalModeTXT = tk.Label(window, text='到院方式:', font=('华文细黑', 12)).place(x=300, y=0)
    InputBox(ArrMode, 400, 0)

    # ArrivalMode = ['步入', '轮椅', '120', '扶入']
    # ArrivalModeLabel = tk.ttk.Combobox(window, values=tuple(ArrivalMode)).place(x=400, y=0, width=60, height=20)

    tk.Label(window, text='主诉(输入):', font=('华文细黑', 12)).place(x=300, y=40)
    name_input = tk.Text(window, width=30, height=3).place(x=400, y=40)

    tk.Label(window, text='主诉(参考):', font=('华文细黑', 12)).place(x=300, y=100)
    name_refer = tk.Text(window, width=30, height=3, wrap=tk.WORD)
    name_refer.insert(tk.END, ChiefComp)
    name_refer.place(x=400, y=100)


    # InputBoxChiefComplaintRef(ChiefComp, 400, 100)
    #*************************************等级和科室
    #*************************************等级和科室
    LevelTXT = tk.Label(window, text='等级:', font=('华文细黑', 16)).place(x=260, y=215)
    LevelS = ['四级', '三级', '二级','一级' ]
    global LevelModeLabel, DepartmentLabel, Level, Department
    LevelModeLabel = tk.ttk.Combobox(window, values=tuple(LevelS), state="readonly")
    LevelModeLabel.place(x=320, y=220, width=70, height=20)
    
    DepartmentTXT = tk.Label(window, text='科室:', font=('华文细黑', 16)).place(x=400, y=215)
    DepartmentS = ['外科','内科','耳鼻喉', '产科','妇科', '眼科', '神经内科', '神经外科', '骨科', '创伤救治中心']
    DepartmentLabel = tk.ttk.Combobox(window, values=tuple(DepartmentS), state="readonly")
    DepartmentLabel.place(x=460, y=220, width=140, height=20)

    # 第7步，#***************************   计数器
    tk.Label(window, text='已完成:', font=('华文细黑', 10)).place(x=180, y=320)
    label = tk.Label(window, text=len(HistoryData),font=('华文细黑','10'))
    label.place(x=230, y=320)

    Parems = ['到院方式', '性别', '出生日期', 'T℃', 'P(次/分)', 'R(次/分)', 'BP(mmHg)', 'SpO2']
    Funcs = ['Arr_way', 'Gender', 'Age', 'Temperature', 'Pulse', 'Respiration', 'BloodPressure', 'SpO2']
    dicLevel = {0: '一级', 1: '二级', 2: '三级', 3: '四级'}
    dicDepart = {0: '产科', 1: '内科', 2: '创伤救治中心', 3: '外科', 4: '妇科', 5: '眼科', 6: '神经内科', 7: '神经外科', 8: '耳鼻喉', 9: '骨科'}

    LIB1 = {idx: i for idx, i in zip(Parems, Funcs)}
    Length = [4, 3, 5, 3, 3, 3, 6, 3]
    LIB2 = {idx: i for idx, i in zip(Parems, Length)}
    model = TransNet()
    weight = r'./weight/TransNet.h5'
    model.load_weights(weight)    

    stopwords = pd.read_csv("./StopWords/common.txt",quoting=3, sep="\t", encoding='utf-8')
    exclude = []
    for i in np.asarray(stopwords):
        exclude.append(i[0])
    exclude.extend(' ')
    segcut = Pkuseg.pkuseg(model_name = "Medicine", postag = False)  # 程序会自动下载所对应的细领域模型  
    
    Level, Department = AITriageModule()
    LevelModeLabel.set(Level)
    DepartmentLabel.set(Department)
    
    start = time.time()
    # 第7步，#*************************************确认 和 下一条 按钮
    btn_ok = tk.Button(window, text='OK', font=('Times New Roman', 15), width=10, height=2)
    btn_ok.place(x=280, y=300)
    btn_next = tk.Button(window, text='Next', font=('Times New Roman', 15), width=10, height=2, command=button_click)
    btn_next.place(x=450, y=300)
    
    # 第10步，主窗口循环显示
    window.mainloop()

请输入分诊人员的姓名（建议中文）：xiaoyi
请输入起始索引(比如从1开始)：1
请输入结束索引（比如到100结束）：100
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
data

In [4]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# author:
import pickle, os, json, argparse, sys, time
from Extkinter import *
from PIL import Image,ImageTk
from datetime import datetime
import pandas as pd
from tkinter import *
import tkinter as tk
from tkinter import ttk
import tkinter.messagebox
import time
import numpy as np
np.seterr(divide='ignore', invalid='ignore')

from myEncoder.Encode import VitalSigns2Digital
import Pkuseg  # 专业领域分词
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

from keras.utils import to_categorical
import tensorflow as tf
from tensorflow.compat.v1 import InteractiveSession
tf.keras.backend.clear_session()
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

from TransNet import TransNet
import pyttsx3
import threading
import warnings
warnings.filterwarnings("ignore")
os.environ["CUDA_VISIBLE_DEVICES"] = "0" or "1"
tok = Tokenizer()
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"


def InputBox(default_value, x, y):
    entry = ttk.Entry(window, width=10)
    entry.place(x=x, y=y)
    entry.insert(0, default_value)


def InputBoxChiefComplaintRef(default_value, x, y):
    entry = ttk.Entry(window, width=30)
    entry.place(x=x, y=y)
    entry.insert(0, default_value)


def AgeTranslation(birth):
    if birth != '空值' and birth != '':
        if "/" in birth:
            birth_d = datetime.strptime(birth, "%Y/%m/%d")
        elif "-" in birth:
            birth_d = datetime.strptime(birth, "%Y-%m-%d").date()
        else:
            birth_d = datetime.strptime(birth, "%Y%m%d").date()
        today_d = datetime.now()
        age = today_d.year - birth_d.year
        return age
    else:
        return '空值'


def update_patient_info():
    global idx, Name, ID, Gender, Age, Temp, PulseRate, Respiration, ReTriage
    global BloodPres, SpO2, ArrMode, ChiefComp, EndIndex, TriageTime

    if idx < EndIndex:
        Name = data['病人姓名'][idx]
        ID = data['病人ID'][idx]
        Gender = data['性别'][idx]
        Age = AgeTranslation(data['出生日期'][idx])
        TriageTime = data['分诊时间'][idx]
        Temp = data['T℃'][idx]
        PulseRate = data['P(次/分)'][idx]
        Respiration = data['R(次/分)'][idx]
        BloodPres = data['BP(mmHg)'][idx]
        SpO2 = data['SpO2'][idx]
        ArrMode = data['到院方式'][idx]
        ChiefComp = data['分诊印象'][idx]
        ReTriage = data['二次分诊'][idx]

        # 更新GUI中的患者信息
        InputBox(Name, 160, 0)
        InputBox(ID, 160, 30)
        InputBox(Gender, 160, 60)
        InputBox(Age, 160, 90)
        InputBox(Temp, 60, 200)
        InputBox(PulseRate, 60, 230)
        InputBox(Respiration, 60, 260)
        InputBox(BloodPres, 60, 290)
        InputBox(SpO2, 60, 320)
        InputBox(ArrMode, 400, 0)
        name_refer.delete(1.0, tk.END)
        name_refer.insert(tk.END, ChiefComp)
        LevelFinal = LevelModeLabel.get()
        DepartmentFinal = DepartmentLabel.get()
    else:
        tk.messagebox.showinfo("提示", "已经到达数据末尾，将关闭窗口。")
        window.destroy()  # 关闭窗口



def creat_csv(file_path, COLUMN):
    if not os.path.exists(file_path):
        tabel = pd.DataFrame(columns=COLUMN)
        tabel.to_csv(file_path, index=False, header=True, encoding="utf_8_sig")
    else:
        tabel = pd.read_csv(file_path, engine='python')
    return tabel


def data_input(self, COLUMN, variables):
    data = pd.DataFrame(columns=COLUMN)
    for idx, i in zip(COLUMN, variables):
        data.at[0, idx] = i
    VS = [np.hstack(([to_categorical(self.func(data[i].values[0], i), LIB2[i]) for i in Parems]))]
    IM = self.unstructure(data)
    Data = np.hstack([VS, IM])
    return [VS], Data

def TIME_TRIAGE(first_time, final_time):
# time_1 = '2020/9/1 12:38'
# time_2 = '2020/9/2 12:44'
    time1 = datetime.strptime(first_time, "%Y/%m/%d %H:%M")
    time2 = datetime.strptime(final_time, "%Y/%m/%d %H:%M")
    seconds = (time2 - time1).total_seconds()
    return seconds

def AcquireIndex(DData, start_ind, end_ind):
    TimeLength = TIME_TRIAGE(DData['分诊时间'][start_ind], DData['分诊时间'][end_ind])
    length_data = end_ind-start_ind
    if TimeLength <= 12*60*60:
        e= DData["病人姓名"][start_ind: end_ind]
        r = np.asarray(e).tolist()
        seen = []
        index = []
        for ind, i in enumerate(r):
            if i not in seen:
                seen.append(i)
                index.append(ind)
    else:
        seen = []
        index = []
        for i in range(length_data-1):
            for j in np.arange(i+1,length_data):
                time_length = TIME_TRIAGE(DData['分诊时间'][i], DData['分诊时间'][j])
            if time_length >= 12*60*60:
                e= DData["病人姓名"][i: j]
                r = np.asarray(e).tolist()
                for ind, k in enumerate(r):
                    if i not in seen:
                        seen.append(k)
                        index.append(ind)
    return index

def ResultFinal(RevisedDictionary, y_predict1, y_predict2):
    if '入抢救室' in RevisedDictionary:
        y_predict1 = '一级'
        RevisedDictionary = ['入抢救室']
    elif len(RevisedDictionary) == 0:
        RevisedDictionary = [y_predict2]
    else:
        RevisedDictionary.append(y_predict2)
        RevisedDictionary = np.unique(RevisedDictionary)
        if len(RevisedDictionary) == 1:
            RevisedDictionary = RevisedDictionary[0]
    return y_predict1, RevisedDictionary


def Conditional_judgment(WhitePath, Chief_Complaint):
    DepartWhiteList = os.listdir(WhitePath)
    Result = []
    for k in DepartWhiteList:
        Path = os.path.join(WhitePath, k)
        whitelist = pd.read_csv(Path, header=None)
        include = []
        for i in whitelist[0]:
            include.append(i)
        if any(word if word in Chief_Complaint else False for word in include):
            Result.append(k[:-4])
        else:
            pass
    return Result


def AITriageModule():
    global idx, ID, Name, Gender, Age, Temp, PulseRate, Respiration
    global BloodPres, SpO2, ArrMode, ChiefComp
    Time = time.strftime('%Y/%m/%d %X', time.localtime())
    Triage_Time = Time
    Variables = [Gender, data['出生日期'][idx], ArrMode, Temp, PulseRate, Respiration, BloodPres, SpO2]
    Variables = ['空值' if str(i).isspace() or len(str(i)) == 0 else i for i in Variables]
    ColumnsTransfer = ['性别', '出生日期', '到院方式', 'T℃', 'P(次/分)', 'R(次/分)', 'BP(mmHg)', 'SpO2']

    data_VSandIM = pd.DataFrame(columns=ColumnsTransfer)
    for ind, i in zip(ColumnsTransfer, Variables):
        data_VSandIM.at[0, ind] = i
    VS = [np.hstack(([to_categorical(func(data_VSandIM[i].values[0], i), LIB2[i]) for i in ColumnsTransfer]))]
    IM = Unstructure(ChiefComp)
    DataVSandIM = np.hstack([VS, IM])
    predictions = model.predict([[VS], DataVSandIM])
    y_predict1 = dicLevel[np.argmax(predictions[0])]
    y_predict2 = dicDepart[np.argmax(predictions[1])]

    RevisedDictionary = Conditional_judgment(WhitePath, ChiefComp)
    Level, Department = ResultFinal(RevisedDictionary, y_predict1, y_predict2)
    return Level, Department


def func(x, Name):
    funcs = LIB1[Name]
    result = getattr(VitalSigns2Digital(), funcs)(x)
    return result


def SegmentFunc(content, stopwords):
    segment = []
    segs = segcut.cut(content)
    for seg in segs:
        if seg not in stopwords:
            segment.append(seg)
    return segment


def Unstructure(ChiefComplaint):
    im = SegmentFunc(ChiefComplaint, exclude)
    tok.fit_on_texts([im])
    with open('./JSON/word_index.json', 'r') as f:
        word_index = json.load(f)
    tok.word_index = word_index
    x = tok.texts_to_sequences([im])
    X = sequence.pad_sequences(x, maxlen=13)
    return X


def button_click_NEXT():
    global counter, idx, Consumption, start, end
    global ID, Name, Gender, Age, Temp, PulseRate, Respiration
    global BloodPres, SpO2, ArrMode, ChiefComp

    SingleData = []
    label.config(text=str(counter))

    Level_Verify = LevelModeLabel.get()
    Department_Verify = DepartmentLabel.get()

    end = time.time()
    Consumption = np.round((end - start), 3)
    start = time.time()

    SingleData = [ID, Name, Gender, data['出生日期'][idx], ArrMode, ChiefComp, Temp, PulseRate,
                  Respiration, BloodPres, SpO2, data['M分诊耗时(秒)'][idx], data['级别'][idx],
                  data['去向'][idx], data['M级别'][idx], data['M科室'][idx], Consumption, Level_Verify, Department_Verify]
    Write(save_path, SingleData, INDEX)
    counter += 1
    
    if counter >= EndIndex - 1:
        tk.messagebox.showinfo("提示", "已经到达数据末尾，将关闭窗口。")
        window.destroy()
    else:
        update_patient_info()
        Level, Department = AITriageModule()
        LevelModeLabel.set(Level)
        DepartmentLabel.set(Department)

def button_click_OK():
    global counter, idx, Level, Department, Consumption, start, end
    HistoryData = pd.read_csv(save_path, engine='python')
    HistoryData['级别预测正确'] = HistoryData['级别'] == HistoryData['级别AI']
    HistoryData['去向预测正确'] = HistoryData['去向'] == HistoryData['科室AI']
    HistoryData['同时正确'] = HistoryData['级别预测正确'] & HistoryData['去向预测正确']
    Time_ = (HistoryData['M分诊耗时(秒)']-HistoryData['AI分诊耗时(秒)'])/HistoryData['M分诊耗时(秒)']
    
    EfficiencyImprovement = (Time_/len(HistoryData)).mean()
    Leve_Score = HistoryData['级别预测正确'].mean()
    Depart_Score = HistoryData['去向预测正确'].mean()
    Whole_Score = HistoryData['同时正确'].mean()
    tk.messagebox.showinfo("分诊结果",
                           f'\n级别预测精度: {100 * Leve_Score:.2f}%\n科室预测精度: {100 * Depart_Score:.2f}% \n效率提升情况: {100 * EfficiencyImprovement:.2f}%')
    #         window.destroy()
    update_patient_info()

def Write(save_path, variables, columns):
    try:
        DATA = pd.read_csv(save_path)
    except FileNotFoundError:
        DATA = pd.DataFrame(columns=columns)
    new_row = pd.Series(variables, index=columns)
    DATA = pd.concat([DATA, new_row.to_frame().transpose()], ignore_index=True)
    DATA.to_csv(save_path, index=False, encoding="utf_8_sig")

def BlackList(blacklistPath):
    blacklist = pd.read_csv(blacklistPath, header=None)
    Exclude = []
    for i in blacklist[0]:
        Exclude.append(i)
    return Exclude

if __name__ == '__main__':
    # 全局变量
    window = None
    data = None
    Name, ID, Gender, Age, Temp, PulseRate, Respiration, BloodPres, SpO2, ArrMode, ChiefComp, LevelFinal, DepartmentFinal = [
                                                                                                                                None] * 13
    name_refer = None  # 声明为全局变量

    path = r"./data/Data.txt"
    NameDoctor = input('请输入分诊人员的姓名（建议中文）：')
    StartIndex = int(input('请输入起始索引(比如从1开始)：'))
    if StartIndex == 0:
        StartIndex = 1
    EndIndex = int(input('请输入结束索引（比如到100结束）：'))
    ResultPath = r'./result'
    if not os.path.exists(ResultPath):
        os.makedirs(ResultPath)
    save_path = os.path.join(ResultPath, NameDoctor + '_TriageData.csv')

    window = tk.Tk()
    window.title('智能分诊系统v1.0-测试版本')
    window.geometry('620x400')
    window.resizable(0, 0)

    WhitePath = r'./StopWords/whitelist'

    Columns = ['病人ID', '病人姓名', '性别', '出生日期', '到院方式', '分诊印象', 'T℃', 'P(次/分)',
               'R(次/分)', 'BP(mmHg)', 'SpO2', 'M分诊耗时(秒)', '级别', '去向', 'M级别', 'M科室', 'AI分诊耗时(秒)', '级别AI', '科室AI']
    RawData = pd.read_csv(path, engine='python')
    RawData = RawData.fillna(value='空值')

    if EndIndex > len(RawData):
        EndIndex = len(RawData)
    blacklistPath = r"./StopWords/Blacklist.txt"
    Exclude = BlackList(blacklistPath)

    length_data = EndIndex - StartIndex
    Sum_Right = AcquireIndex(RawData, StartIndex, EndIndex - 1)
    RightData = [index_re for index_re, _ in enumerate(RawData["二次分诊"][:length_data]) if
                 RawData["二次分诊"][:length_data][index_re] == '是']
    ObjIndex = np.unique(list(Sum_Right) + list(RightData))  # 分诊正确

    First_index = []
    ObjData = []
    for k in ObjIndex:
        ChiefComplaint = RawData['分诊印象'][k]
        if all(word if word not in ChiefComplaint else False for word in Exclude) and ChiefComplaint != '空值':
            First_index.append(k)

    list_class = ['病人ID', '病人姓名', '性别', '出生日期', '分诊时间', '二次分诊', '到院方式', '分诊印象', 'T℃', 'P(次/分)',
                  'R(次/分)', 'BP(mmHg)', 'SpO2', 'M分诊耗时(秒)', 'M级别', 'M科室', '去向', '级别']
    for p in First_index:
        ObjData.append([RawData[q][p] for q in list_class])
    data = pd.DataFrame(ObjData, columns=list_class)

    data = data.fillna(value='空值')
    Tabel = creat_csv(save_path, Columns)
    INDEX = Columns
    HistoryData = pd.read_csv(save_path, engine='python')
    if   StartIndex > len(HistoryData):
        counter = StartIndex
    elif StartIndex<len(HistoryData)<=EndIndex:
        counter = len(HistoryData)
    elif StartIndex>0 and EndIndex <= len(HistoryData):
        # tk.messagebox.showinfo("提示", "该部分已处理过，将关闭窗口。")
        # window.destroy()
        print('该部分已处理过！！')
        os._exit(0)# 括号要放值os._exit(0)--正常退出
    else:
        counter = 0
    idx = counter

    Name = data['病人姓名'][idx]
    ID = data['病人ID'][idx]
    Gender = data['性别'][idx]
    Age = AgeTranslation(data['出生日期'][idx])

    Temp = data['T℃'][idx]
    PulseRate = data['P(次/分)'][idx]
    Respiration = data['R(次/分)'][idx]
    BloodPres = data['BP(mmHg)'][idx]
    SpO2 = data['SpO2'][idx]
    ArrMode = data['到院方式'][idx]
    ChiefComp = data['分诊印象'][idx]
    name_refer = tk.Text(window, width=30, height=3, wrap=tk.WORD)
    name_refer.insert(tk.END, ChiefComp)
    name_refer.place(x=400, y=100)

    # *************************************患者信息***********************
    # *************************************患者信息***********************
    tk.Label(window, text="", bg="lightgray", fg="white").place(x=0, y=0, width=250, height=140)
    tk.Label(window, text="患者信息", bg="lightblue", fg="white", font=("华文细黑", 12)).place(x=15, y=165)

    # 第4步，加载 wellcome image
    image_text = Text(window, height=10, width=16)
    photo = PhotoImage(file="RC.png")
    image_text.image_create('0.0', image=photo)
    image_text.grid(row=2, column=1, columnspan=2, sticky=W)

    # 第4步，加载 校LOGO
    photo_SEU = PhotoImage(file="./JSON/Logo/SEUlogo.png")
    label_SEU = Label(window, image=photo_SEU)
    #     label_SEU.place(x=0, y=360, width=160, height=100)
    label_SEU.place(x=0, y=360)

    # 第5步
    tk.Label(window, text='姓名:', bg="lightgray", font=('华文细黑', 12)).place(x=120, y=0)
    InputBox(Name, 160, 0)
    tk.Label(window, text='ID:', bg="lightgray", font=('华文细黑', 12)).place(x=120, y=30)
    InputBox(ID, 160, 30)
    tk.Label(window, text='性别:', bg="lightgray", font=('华文细黑', 12)).place(x=120, y=60)
    InputBox(Gender, 160, 60)
    # # 单选钮，男
    # def show():
    #     print(Gender[sex.get()-1][0])
    # sex = tk.IntVar(value=1)
    # Gender = [("男", 0),("女", 1),]
    # radioMan = tk.Radiobutton(window, variable=sex, value=1, text='男', bg="lightgray", font=('华文细黑', 12),command=show)
    # radioMan.place(x=160, y=60, width=50, height=20)
    # radioWoman = tk.Radiobutton(window, variable=sex, value=2, text='女', bg="lightgray", font=('华文细黑', 12),command=show)
    # radioWoman.place(x=210, y=60, width=70, height=20)

    tk.Label(window, text='年龄:', bg="lightgray", font=('华文细黑', 12)).place(x=120, y=90)
    InputBox(Age, 160, 90)

    # 第6步，*************************************生命体征
    # 第6步，*************************************生命体征
    tk.Label(window, text="", bg="darkseagreen", fg="black").place(x=0, y=160, width=160, height=200)
    tk.Label(window, text="生命体征", bg="olivedrab", fg="white", font=("华文细黑", 12)).place(x=0, y=165)
    tk.Label(window, text='体温:', bg="darkseagreen", fg="black", font=('华文细黑', 12)).place(x=15, y=200)
    InputBox(Temp, 60, 200)
    tk.Label(window, text='心率:', bg="darkseagreen", fg="black", font=('华文细黑', 12)).place(x=15, y=230)
    InputBox(PulseRate, 60, 230)
    tk.Label(window, text='呼吸:', bg="darkseagreen", fg="black", font=('华文细黑', 12)).place(x=15, y=260)
    InputBox(Respiration, 60, 260)
    tk.Label(window, text='血压:', bg="darkseagreen", fg="black", font=('华文细黑', 12)).place(x=15, y=290)
    InputBox(BloodPres, 60, 290)
    tk.Label(window, text='血氧:', bg="darkseagreen", fg="black", font=('华文细黑', 12)).place(x=15, y=320)
    InputBox(SpO2, 60, 320)

    # *************************************到院方式
    # *************************************到院方式
    ArrivalModeTXT = tk.Label(window, text='到院方式:', font=('华文细黑', 12)).place(x=300, y=0)
    InputBox(ArrMode, 400, 0)

    # ArrivalMode = ['步入', '轮椅', '120', '扶入']
    # ArrivalModeLabel = tk.ttk.Combobox(window, values=tuple(ArrivalMode)).place(x=400, y=0, width=60, height=20)

    tk.Label(window, text='主诉(输入):', font=('华文细黑', 12)).place(x=300, y=40)
    name_input = tk.Text(window, width=30, height=3).place(x=400, y=40)

    tk.Label(window, text='主诉(参考):', font=('华文细黑', 12)).place(x=300, y=100)
    name_refer = tk.Text(window, width=30, height=3, wrap=tk.WORD)
    name_refer.insert(tk.END, ChiefComp)
    name_refer.place(x=400, y=100)

    # InputBoxChiefComplaintRef(ChiefComp, 400, 100)
    # *************************************等级和科室
    # *************************************等级和科室
    LevelTXT = tk.Label(window, text='等级:', font=('华文细黑', 16)).place(x=260, y=215)
    LevelS = ['四级', '三级', '二级', '一级']
    global LevelModeLabel, DepartmentLabel, Level, Department
    LevelModeLabel = tk.ttk.Combobox(window, values=tuple(LevelS), state="readonly")
    LevelModeLabel.place(x=320, y=220, width=70, height=20)

    DepartmentTXT = tk.Label(window, text='科室:', font=('华文细黑', 16)).place(x=400, y=215)
    DepartmentS = ['外科', '内科', '耳鼻喉', '产科', '妇科', '眼科', '神经内科', '神经外科', '骨科', '创伤救治中心']
    DepartmentLabel = tk.ttk.Combobox(window, values=tuple(DepartmentS), state="readonly")
    DepartmentLabel.place(x=460, y=220, width=140, height=20)

    # 第7步，#***************************   计数器
    tk.Label(window, text='已完成:', font=('华文细黑', 10)).place(x=180, y=320)
    label = tk.Label(window, text=len(HistoryData), font=('华文细黑', '10'))
    label.place(x=230, y=320)

    Parems = ['到院方式', '性别', '出生日期', 'T℃', 'P(次/分)', 'R(次/分)', 'BP(mmHg)', 'SpO2']
    Funcs = ['Arr_way', 'Gender', 'Age', 'Temperature', 'Pulse', 'Respiration', 'BloodPressure', 'SpO2']
    dicLevel = {0: '一级', 1: '二级', 2: '三级', 3: '四级'}
    dicDepart = {0: '产科', 1: '内科', 2: '创伤救治中心', 3: '外科', 4: '妇科', 5: '眼科', 6: '神经内科', 7: '神经外科', 8: '耳鼻喉', 9: '骨科'}

    LIB1 = {idx: i for idx, i in zip(Parems, Funcs)}
    Length = [4, 3, 5, 3, 3, 3, 6, 3]
    LIB2 = {idx: i for idx, i in zip(Parems, Length)}
    model = TransNet()
    weight = r'./weight/TransNet.h5'
    model.load_weights(weight)

    stopwords = pd.read_csv("./StopWords/common.txt", quoting=3, sep="\t", encoding='utf-8')
    exclude = []
    for i in np.asarray(stopwords):
        exclude.append(i[0])
    exclude.extend(' ')
    segcut = Pkuseg.pkuseg(model_name="Medicine", postag=False)  # 程序会自动下载所对应的细领域模型

    Level, Department = AITriageModule()
    LevelModeLabel.set(Level)
    DepartmentLabel.set(Department)

    start = time.time()
    # 第7步，#*************************************确认 和 下一条 按钮
    btn_ok = tk.Button(window, text='OK', font=('Times New Roman', 15), width=10, height=2, command=button_click_OK)
    btn_ok.place(x=280, y=300)
    btn_next = tk.Button(window, text='Next', font=('Times New Roman', 15), width=10, height=2, command=button_click_NEXT)
    btn_next.place(x=450, y=300)

    # 第10步，主窗口循环显示
    window.mainloop()

请输入分诊人员的姓名（建议中文）：xiaoyi
请输入起始索引(比如从1开始)：1
请输入结束索引（比如到100结束）：100


In [11]:
global counter, idx, Level, Department, Consumption, start, end
HistoryData = pd.read_csv(save_path, engine='python')
Time_ = (HistoryData['M分诊耗时(秒)']-HistoryData['AI分诊耗时(秒)'])/HistoryData['M分诊耗时(秒)']
EfficiencyImprovement = (Time_/len(HistoryData)).mean()
print("分诊结果\n", f'\n级别预测精度: {100 * Leve_Score:.2f}%\n科室预测精度: {100 * Depart_Score:.2f}% \n效率提升情况: {100 * EfficiencyImprovement:.2f}%')


分诊结果
 
级别预测精度: 88.89%
科室预测精度: 83.33% 
效率提升情况: 0.27%


In [10]:
def button_click_OK():
    global counter, idx, Level, Department, Consumption, start, end
    HistoryData = pd.read_csv(save_path, engine='python')
    HistoryData['级别预测正确'] = HistoryData['级别'] == HistoryData['级别AI']
    HistoryData['去向预测正确'] = HistoryData['去向'] == HistoryData['科室AI']
    HistoryData['同时正确'] = HistoryData['级别预测正确'] & HistoryData['去向预测正确']
    Time_ = (HistoryData['M分诊耗时(秒)']-HistoryData['AI分诊耗时(秒)'])/HistoryData['M分诊耗时(秒)']
    
    EfficiencyImprovement = (Time_/len(HistoryData)).mean()
    Leve_Score = HistoryData['级别预测正确'].mean()
    Depart_Score = HistoryData['去向预测正确'].mean()
    Whole_Score = HistoryData['同时正确'].mean()
    tk.messagebox.showinfo("分诊结果",
                           f'\n级别预测精度: {100 * Leve_Score:.2f}%\n科室预测精度: {100 * Depart_Score:.2f}% \n效率提升情况: {100 * EfficiencyImprovement:.2f}%')
    #         window.destroy()
    update_patient_info()

0.002656901901860083